# 004 Simple Database Queries
This notebook demonstrates basic SQL queries against the `assets.db` database to explore stock metadata and price data.

In [3]:
import sys
import pandas as pd
import sqlite3
import os
from pathlib import Path

# Define BASE_DIR and adjust sys.path before importing
BASE_DIR = Path(os.getcwd()).parent  # Points to stat_656_autotrader/ from Notebooks/
sys.path.append(str(BASE_DIR))
print(f"Project root added to sys.path: {BASE_DIR}")

from src.config import BASE_DIR, DB_DIR  # Import from config.py
from src.utils.db_utils import get_db_connection

# Set display options for pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

Project root added to sys.path: d:\dev\stat_656_autotrader


In [4]:
def execute_sql_query(query, params=None, db_path=str(DB_DIR / 'assets.db'), return_df=True):
    """
    Execute an SQL query against the database and return results.
    
    Args:
        query (str): SQL query to execute.
        params (tuple or list, optional): Parameters for the query (e.g., for WHERE clauses).
        db_path (str): Path to assets.db, defaults to DB_DIR/assets.db.
        return_df (bool): If True, return a pandas DataFrame; if False, return raw rows and columns.
    
    Returns:
        pd.DataFrame or (list, list): Query results as a DataFrame or (rows, column names) if return_df=False.
    """
    try:
        conn = get_db_connection(db_name=db_path)
        if return_df:
            df = pd.read_sql_query(query, conn, params=params)
            conn.close()
            print(f"Query returned {len(df)} rows")
            return df
        else:
            cursor = conn.cursor()
            cursor.execute(query, params or [])
            rows = cursor.fetchall()
            columns = [desc[0] for desc in cursor.description]
            conn.close()
            print(f"Query returned {len(rows)} rows")
            return rows, columns
    except sqlite3.Error as e:
        print(f"Database error: {e}")
        return pd.DataFrame() if return_df else ([], [])

In [7]:
# Test with a simple query
test_query = "SELECT symbol, name FROM asset_metadata LIMIT 5"
test_result = execute_sql_query(test_query)
test_result

Connecting to database: D:\dev\stat_656_autotrader\databases\assets.db
Query returned 5 rows


,symbol,name
0,CCLDO,"CareCloud, Inc. 8.75% Series B Cumulative Rede..."
1,BHVN,Biohaven Ltd.
2,ZUMZ,Zumiez Inc. Common Stock
3,EMR,Emerson Electric Co.
4,CCL,Carnival Corporation


# Example 1: Fetch All Active Stock Tickers

In [5]:
active_tickers_query = """
    SELECT asset_id, symbol, name, exchange
    FROM asset_metadata
    WHERE asset_type = 'us_equity' AND is_active = 1
    ORDER BY symbol
"""
active_tickers = execute_sql_query(active_tickers_query)
active_tickers.head(2)

Connecting to database: D:\dev\stat_656_autotrader\databases\assets.db
Query returned 7462 rows


,asset_id,symbol,name,exchange
0,7324,A,Agilent Technologies Inc.,NYSE
1,7247,AA,Alcoa Corporation,NYSE


# Example 2: Get the Latest Price for Each Stock

In [ ]:
latest_prices_query = """
    SELECT am.symbol, ap.date, ap.close, ap.fetched_at
    FROM asset_prices ap
    JOIN asset_metadata am ON ap.asset_id = am.asset_id
    WHERE am.asset_type = 'Stock' AND am.is_active = 1
    AND ap.date = (
        SELECT MAX(date)
        FROM asset_prices ap2
        WHERE ap2.asset_id = ap.asset_id
    )
    ORDER BY am.symbol
"""
latest_prices = execute_sql_query(latest_prices_query)
latest_prices.head(2)

Connecting to database: D:\dev\stat_656_autotrader\databases\assets.db
